In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup

# Set up the Selenium WebDriver (make sure you have the appropriate browser driver installed)
driver = webdriver.Chrome()

url = "https://anilist.co/search/anime/top-100"
driver.get(url)

# Scroll to the bottom of the page
def scroll_to_bottom():
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight*0.6);")


# Wait for the page to load
def wait_for_page_to_load():
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div.ptitle-internal a.tileLink")))

# Perform the scrolling and extraction
scroll_to_bottom()
wait_for_page_to_load()

# Extract the information you need from the scrolled page
# actor_elements = driver.find_elements(By.CSS_SELECTOR, "div.ptitle-internal a.tileLink")
# for actor_element in actor_elements:
#     name = actor_element.text.strip()
#     print(name)

# Repeat the process
count=0
anime_name=[]
while True:
    scroll_to_bottom()
    wait_for_page_to_load()

    # Extract the information from the newly loaded content
    html_response = driver.page_source

# Create a BeautifulSoup object to parse the HTML
    soup = BeautifulSoup(html_response, 'html.parser')

# Find all anchor tags with the class "title"
    anchor_tags = soup.find_all('a', class_='title')

# Extract the URLs and store them in a list
    urls = [tag['href'] for tag in anchor_tags]

    print(urls)
        
    # Check if the end of the page is reached
    if len(urls) == 0:
        break

# Close the browser
driver.quit()


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Set up the Selenium WebDriver
driver = webdriver.Chrome()
driver.get('https://anilist.co/search/anime/top-100')

# Calculate the target scroll position as a percentage of the entire document's height
scroll_target = driver.execute_script("return document.documentElement.scrollHeight * 50")

# Scroll until reaching the target height
while True:
    # Scroll the page by a small amount
    driver.execute_script("window.scrollBy(0, 100);")
    
    # Wait for the new content to load (you may need to adjust the sleep time)
    time.sleep(0.5)
    
    # Check if reaching the target height
    current_scroll = driver.execute_script("return window.pageYOffset;")
    if current_scroll >= scroll_target:
        break

# Get the HTML source after scrolling to the target height
html_response = driver.page_source

# Create a BeautifulSoup object to parse the HTML
soup = BeautifulSoup(html_response, 'html.parser')

# Find all anchor tags with the class "title"
anchor_tags = soup.find_all('a', class_='title')

# Extract the URLs and store them in a list
urls = [tag['href'] for tag in anchor_tags]

print(urls)

# Close the WebDriver
driver.quit()


In [ ]:
len(urls)

In [ ]:
import pandas as pd
df = pd.DataFrame(urls, columns=['URL'])

# Save the DataFrame to a CSV file
df.to_csv('urls.csv', index=False)

In [ ]:
import pandas as pd
df = pd.read_csv("urls.csv")
urls = df['URL'].values
links=list(urls)

In [ ]:
import mechanicalsoup
import random

headers = {
    "user-agent": random.choice([
        "Mozilla/5.0 (iPhone; CPU iPhone OS 8_0 like Mac OS X) AppleWebKit/537.51.1 (KHTML, like Gecko) Version/8.0 Mobile/11A465 Safari/9537.53",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 15_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) GSA/98.0.300355889 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 16_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) GSA/270.0.542728515 Mobile/15E148 Safari/604.1",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 16_3 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 it.fanpage.rn.ios/3.10.9",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 16_5_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 MobileIron/2.18.1 Version/16.5.1 Safari/605.1.15",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1_2 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Mobile/15E148 MobileIron/2.18.1 Version/16.1.2 Safari/605.1.15",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 12_4_8 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/81.0.4044.113 Mobile/16G201 Safari/604.1",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 12_5_7 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/13.0.5 Mobile/15E148 Safari/604.1 RDDocuments/7.5.4.747",
        "Mozilla/5.0 (iPhone; CPU iPhone OS 16_5 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/87.0.4280.163 Mobile/15E148 Safari/604.1",
    ]),
    "Accept-Language": "en-US,en;q=0.5"
}

browser = mechanicalsoup.StatefulBrowser()

for url in urls:
    url=f"https://anilist.co{url}characters"
    browser.open(url, headers=headers)
    soup = browser.get_current_page()

    try:
        h2_tag = soup.select_one('h2[data-testid="subtitle"]')

        # Extract the text content of the h2 tag
        if h2_tag:
            subtitle = h2_tag.text
            print("Name:", subtitle)
        else:
            print("Subtitle not found.")

        height_li = soup.find('li', id='height')

        # Extract the height value
        if height_li:
            height_div = height_li.find('div', class_='ipc-html-content-inner-div')
            height_text = height_div.get_text(strip=True)
            print("Height:", height_text)
        else:
            print("Height not found.")
    except Exception as e:
        print("An error occurred:", str(e))


In [ ]:
import mechanicalsoup
from bs4 import BeautifulSoup
import random

browser = mechanicalsoup.StatefulBrowser()
url = 'https://anilist.co/anime/114129/Gintama-THE-FINAL/'
response = browser.open(url)
html = response.content

soup = BeautifulSoup(html, 'html.parser')

character_divs = soup.find_all('div', class_='character')
# print(character_divs)
links = [a['href'] for char in character_divs for a in char.select('a[href]')]
print(links)



In [ ]:
import mechanicalsoup
from bs4 import BeautifulSoup

browser = mechanicalsoup.StatefulBrowser()
url = 'https://anilist.co/anime/114129/Gintama-THE-FINAL/'
response = browser.open(url)
html = response.content

# Parse the initial HTML
soup = BeautifulSoup(html, 'html.parser')

# Set the total number of pages (for demonstration purposes)
total_pages = 5

# Iterate through the remaining pages
for page_number in range(2, total_pages + 1):
    # Make a request to the next page URL
    next_url = f'https://anilist.co/anime/114129/Gintama-THE-FINAL/{page_number}/'
    response = browser.open(next_url)
    next_html = response.content

    # Parse the HTML of the next page
    next_soup = BeautifulSoup(next_html, 'html.parser')

    # Extract the relevant content from the next page and append it to the existing soup object
    character_divs = next_soup.find_all('div', class_='character')
    soup.extend(character_divs)

# Extract the href links from the final soup object
links = [a['href'] for char in soup.find_all('div', class_='character') for a in char.select('a[href]')]
print(links)


In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

# Set up the Selenium WebDriver
driver = webdriver.Chrome()
driver.get('https://anilist.co/anime/114129/Gintama-THE-FINAL/characters')

def scroll_to_bottom():
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight*0.6);")
# Calculate the target scroll position as a percentage of the entire document's height
scroll_target = driver.execute_script("return document.documentElement.scrollHeight * 2")

# Scroll until reaching the target height
while True:
    # Scroll the page by a small amount
    driver.execute_script("window.scrollBy(0, 100);")
    
    # Wait for the new content to load (you may need to adjust the sleep time)
    time.sleep(0.5)
    
    # Check if reaching the target height
    current_scroll = driver.execute_script("return window.pageYOffset;")
    
    

# Get the HTML source after scrolling to the target height
html_response = driver.page_source

# Create a BeautifulSoup object to parse the HTML
soup = BeautifulSoup(html, 'html.parser')

character_divs = soup.find_all('div', class_='character')
# print(character_divs)
links = [a['href'] for char in character_divs for a in char.select('a[href]')]

print(links)
# Close the WebDriver
driver.quit()


In [ ]:
import re

ids = [re.search(r'\d+', link).group() for link in links]
ids

In [ ]:
import requests
import json
import time

url = 'https://graphql.anilist.co'

query = '''
query media($id: Int, $page: Int) {
  Media(id: $id) {
    id
    characters(page: $page, sort: [ROLE, RELEVANCE, ID]) {
      pageInfo {
        total
        perPage
        currentPage
        lastPage
        hasNextPage
      }
      edges {
        id
        role
        name
        voiceActorRoles(sort: [RELEVANCE, ID]) {
          roleNotes
          dubGroup
          voiceActor {
            id
            name {
              userPreferred
            }
            language: languageV2
            image {
              large
            }
          }
        }
        node {
          id
          name {
            userPreferred
          }
          image {
            large
          }
        }
      }
    }
  }
}
'''

id = 114129  # Replace with your desired ID

payload = {
    'query': query,
    'variables': {}
}

headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}

has_next_page = True
page = 1
characters_data = []

while has_next_page:
    payload['variables']['id'] = id
    payload['variables']['page'] = page

    response = requests.post(url, json=payload, headers=headers)
    data = response.json()

    # Process the received data here
    media_data = data['data']['Media']
    pageInfo = media_data['characters']['pageInfo']
    edges = media_data['characters']['edges']

    # Append the characters data to the list
    characters_data.extend(edges)

    has_next_page = pageInfo['hasNextPage']
    page += 1

    # Delay between each request to respect rate limit
    time.sleep(1)  # Adjust the delay time as needed

# Save the data to a JSON file
filename = f'characters_data_{id}.json'
with open(filename, 'w') as json_file:
    json.dump(characters_data, json_file, indent=2)

print(f'Received data for ID: {id}. Data saved to {filename}')


In [ ]:
import requests
import json
import time

url = 'https://graphql.anilist.co'

query = '''
query media($id: Int, $page: Int) {
  Media(id: $id) {
    id
    characters(page: $page, sort: [ROLE, RELEVANCE, ID]) {
      pageInfo {
        total
        perPage
        currentPage
        lastPage
        hasNextPage
      }
      edges {
        id
        role
        name
        voiceActorRoles(sort: [RELEVANCE, ID]) {
          roleNotes
          dubGroup
          voiceActor {
            id
            name {
              userPreferred
            }
            language: languageV2
            image {
              large
            }
          }
        }
        node {
          id
          name {
            userPreferred
          }
          image {
            large
          }
        }
      }
    }
  }
}
'''


payload = {
    'query': query,
    'variables': {}
}

headers = {
    'Content-Type': 'application/json',
    'Accept': 'application/json',
}

all_data = []  # List to store all the received data

for id in ids:
    try:
        has_next_page = True
        page = 1
        characters_data = []

        while has_next_page:
            payload['variables']['id'] = id
            payload['variables']['page'] = page

            response = requests.post(url, json=payload, headers=headers)
            data = response.json()

            # Process the received data here
            media_data = data['data']['Media']
            pageInfo = media_data['characters']['pageInfo']
            edges = media_data['characters']['edges']

            # Append the characters data to the list
            characters_data.extend(edges)

            has_next_page = pageInfo['hasNextPage']
            page += 1

            # Delay between each request to respect rate limit
             # Adjust the delay time as needed

        # Append the data for the current ID to the overall data list
        all_data.append({
            'id': id,
            'characters': characters_data
        })

        print(f'Received data for ID: {id}')

    except Exception as e:
        print(f'Error occurred for ID: {id}')
        print(f'Error message: {str(e)}')
        continue

# Save all the data to a JSON file
with open('characters_data.json', 'w') as json_file:
    json.dump(all_data, json_file, indent=2)

print('All requests completed. Data saved to characters_data.json')


In [1]:
## getting data
import pandas as pd

df = pd.read_csv("anime.csv")
df['url'] = df["Node ID"].astype(str) + "/" + df["Node Name"].str.replace(" ", "-")
df

,ID,Node ID,Node Name,url
0,114129,672,Gintoki Sakata,672/Gintoki-Sakata
1,114129,673,Shinpachi Shimura,673/Shinpachi-Shimura
2,114129,674,Kagura,674/Kagura
3,114129,2948,Shinsuke Takasugi,2948/Shinsuke-Takasugi
4,114129,1533,Kotarou Katsura,1533/Kotarou-Katsura
...,...,...,...,...
27729,1946,19896,Count Waltmüller,19896/Count-Waltmüller
27730,1577,3726,Miyuki Kobayakawa,3726/Miyuki-Kobayakawa
27731,1577,3727,Natsumi Tsujimoto,3727/Natsumi-Tsujimoto
27732,1577,3728,Ken Nakajima,3728/Ken-Nakajima


In [3]:
urls = df['url'].values
urls = list(urls)
len(urls)

27734

In [ ]:
import requests
from bs4 import BeautifulSoup
import re
import json

results = []  # List to store the extracted data

for url_ in urls:
    url = f"https://anilist.co/character/{url_}"
    res = requests.get(url)
    soup = BeautifulSoup(res.content, "html.parser")
    des = soup.select(".description-wrap")
    if des:
        des = des[0]
    name_element = soup.find('h1', {'class': 'name'})
    name = name_element.text.strip() if name_element else None
    # Extract data
    birthday = des.find('span', string='Birthday:')
    birthday = birthday.next_sibling.strip() if birthday else None
    
    age = des.find('span', string='Age:')
    age = age.next_sibling.strip() if age else None
    
    gender = des.find('span', string='Gender:')
    gender = gender.next_sibling.strip() if gender else None
    
    blood_type = des.find('span', string='Blood Type:')
    blood_type = blood_type.next_sibling.strip() if blood_type else None
    
    height = des.find('div', string=re.compile(r'Height: \d+ cm'))
    height = height.get_text(strip=True).split(':')[1].split('cm')[0].strip() if height else None
    
    # Store the data as a dictionary
    data = {
        'name': name,
        'Birthday': birthday,
        'Age': age,
        'Gender': gender,
        'Blood Type': blood_type,
        'Height': height
    }
    print(data)
    
    results.append(data)

# Save the results to a JSON file
with open('scraped_data.json', 'w') as file:
    json.dump(results, file)

print("Data saved to scraped_data.json")


In [ ]:
urls

In [5]:
import requests
from bs4 import BeautifulSoup
import time
import re
import json
from concurrent.futures import ThreadPoolExecutor

results = []  # List to store the extracted data

def scrape_data(url):
    try:
        link = f"https://anilist.co/character/{url}"
        res = requests.get(link)
        res.raise_for_status()  # Raise an exception if the request fails

        soup = BeautifulSoup(res.content, "html.parser")
        des = soup.select(".description-wrap")
        if des:
            des = des[0]
        name_element = soup.find('h1', {'class': 'name'})
        name = name_element.text.strip() if name_element else None
        # Extract data
        birthday = des.find('span', string='Birthday:')
        birthday = birthday.next_sibling.strip() if birthday else None

        age = des.find('span', string='Age:')
        age = age.next_sibling.strip() if age else None

        gender = des.find('span', string='Gender:')
        gender = gender.next_sibling.strip() if gender else None

        blood_type = des.find('span', string='Blood Type:')
        blood_type = blood_type.next_sibling.strip() if blood_type else None

        height = des.find('div', string=re.compile(r'Height: \d+ cm'))
        height = height.get_text(strip=True).split(':')[1].split('cm')[0].strip() if height else None
        
        # Store the data as a dictionary
        data = {
            'name': name,
            'Birthday': birthday,
            'Age': age,
            'Gender': gender,
            'Blood Type': blood_type,
            'Height': height
        }
        print(data)

        results.append(data)
    except (requests.RequestException, ValueError) as e:
        print(f"An error occurred while processing {url}: {e}")

# List of URLs to scrape

# Create a ThreadPoolExecutor with maximum concurrency of 5
with ThreadPoolExecutor(max_workers=1) as executor:
    # Submit tasks to the executor
    executor.map(scrape_data, urls)

# Save the results to a JSON file
with open('animefinal.json', 'w') as file:
    json.dump(results, file)

print("Data saved to scraped_data.json")


{'name': 'Gintoki Sakata', 'Birthday': 'Oct 10', 'Age': '27', 'Gender': 'Male', 'Blood Type': 'B', 'Height': '177'}
{'name': 'Shinpachi Shimura', 'Birthday': 'Aug 12', 'Age': '16-18', 'Gender': 'Male', 'Blood Type': None, 'Height': '166'}
{'name': 'Kagura', 'Birthday': 'Nov 3', 'Age': None, 'Gender': 'Female', 'Blood Type': None, 'Height': '155'}
{'name': 'Shinsuke Takasugi', 'Birthday': 'Aug 10', 'Age': '20s', 'Gender': 'Male', 'Blood Type': None, 'Height': '170'}
{'name': 'Kotarou Katsura', 'Birthday': 'Jun 26', 'Age': '20s', 'Gender': 'Male', 'Blood Type': 'AB', 'Height': '175'}
{'name': 'Shouyou Yoshida', 'Birthday': 'Aug 4', 'Age': None, 'Gender': 'Male', 'Blood Type': None, 'Height': None}
{'name': 'Tae Shimura', 'Birthday': 'Oct 31', 'Age': '18', 'Gender': 'Female', 'Blood Type': None, 'Height': '168'}
{'name': 'Sadaharu', 'Birthday': 'Feb 25', 'Age': None, 'Gender': 'Male', 'Blood Type': None, 'Height': '170'}
{'name': 'Tama', 'Birthday': 'Feb 23', 'Age': None, 'Gender': 'Femal

In [3]:
import requests
from bs4 import BeautifulSoup
import time
import re
import json
from concurrent.futures import ThreadPoolExecutor

results = []  # List to store the extracted data
count = 1
request_count = 0
start_time = time.time()

def rate_limiter():
    global request_count, start_time
    elapsed_time = time.time() - start_time

    # Check if the elapsed time is less than a minute and the request count is already 90
    if elapsed_time < 60 and request_count >= 60:
        sleep_time = 60 - elapsed_time  # Sleep for the remaining seconds in the minute
        time.sleep(sleep_time)  # Sleep to adhere to the rate limit
        start_time = time.time()  # Reset the start time
        request_count = 0  # Reset the request count

def scrape_data(url):
    global count, request_count
    rate_limiter()  # Enforce rate limit

    try:
        link = f"https://anilist.co/character/{url}"
        res = requests.get(link)
        res.raise_for_status()  # Raise an exception if the request fails

        soup = BeautifulSoup(res.content, "html.parser")
        des = soup.select(".description-wrap")
        if des:
            des = des[0]
        name_element = soup.find('h1', {'class': 'name'})
        name = name_element.text.strip() if name_element else None
        # Extract data
        birthday = des.find('span', string='Birthday:')
        birthday = birthday.next_sibling.strip() if birthday else None

        age = des.find('span', string='Age:')
        age = age.next_sibling.strip() if age else None

        gender = des.find('span', string='Gender:')
        gender = gender.next_sibling.strip() if gender else None

        blood_type = des.find('span', string='Blood Type:')
        blood_type = blood_type.next_sibling.strip() if blood_type else None

        height = des.find('div', string=re.compile(r'Height: \d+ cm'))
        height = height.get_text(strip=True).split(':')[1].split('cm')[0].strip() if height else None
        count += 1
        # Store the data as a dictionary
        data = {
            'name': name,
            'Birthday': birthday,
            'Age': age,
            'Gender': gender,
            'Blood Type': blood_type,
            'Height': height
        }
        print(data)

        results.append(data)

        request_count += 1  # Increase the request count
    except (requests.RequestException, ValueError) as e:
        print(f"An error occurred while processing {url}: {e}")

# List of URLs to scrape

# Create a ThreadPoolExecutor with maximum concurrency of 5
with ThreadPoolExecutor(max_workers=1) as executor:
    # Submit tasks to the executor
    executor.map(scrape_data, urls)

# Save the results to a JSON file
with open('dataanime.json', 'w') as file:
    json.dump(results, file)

print("Data saved to scraped_data.json")


An error occurred while processing 673/Shinpachi-Shimura: 429 Client Error: Too Many Requests for url: https://anilist.co/character/673/Shinpachi-Shimura
An error occurred while processing 672/Gintoki-Sakata: 429 Client Error: Too Many Requests for url: https://anilist.co/character/672/Gintoki-Sakata
An error occurred while processing 2948/Shinsuke-Takasugi: 429 Client Error: Too Many Requests for url: https://anilist.co/character/2948/Shinsuke-Takasugi
An error occurred while processing 674/Kagura: 429 Client Error: Too Many Requests for url: https://anilist.co/character/674/Kagura
An error occurred while processing 1533/Kotarou-Katsura: 429 Client Error: Too Many Requests for url: https://anilist.co/character/1533/Kotarou-Katsura
An error occurred while processing 28144/Shouyou-Yoshida: 429 Client Error: Too Many Requests for url: https://anilist.co/character/28144/Shouyou-Yoshida
An error occurred while processing 2944/Tae-Shimura: 429 Client Error: Too Many Requests for url: https:

An error occurred while processing 121103/Chika-Fujiwara: 429 Client Error: Too Many Requests for url: https://anilist.co/character/121103/Chika-Fujiwara
An error occurred while processing 125886/Miko-Iino: 429 Client Error: Too Many Requests for url: https://anilist.co/character/125886/Miko-Iino
An error occurred while processing 36309/Narrator: 429 Client Error: Too Many Requests for url: https://anilist.co/character/36309/Narrator
An error occurred while processing 121104/Ai-Hayasaka: 429 Client Error: Too Many Requests for url: https://anilist.co/character/121104/Ai-Hayasaka
An error occurred while processing 126144/Kei-Shirogane: 429 Client Error: Too Many Requests for url: https://anilist.co/character/126144/Kei-Shirogane
An error occurred while processing 126986/Erika-Kose: 429 Client Error: Too Many Requests for url: https://anilist.co/character/126986/Erika-Kose
An error occurred while processing 134326/Tsubame-Koyasu: 429 Client Error: Too Many Requests for url: https://anili